In [ ]:
import pyodbc
import pandas as pd
import shutil
import re
import numpy as np

import time
from datetime import datetime
current_date = datetime.now()
formatted_date = current_date.strftime('%d%m%Y')

def copy_database(original_path, temp_path):
    shutil.copyfile(original_path, temp_path)

def export_access_to_dataframes(database_path):
    # Connection string for Access database
    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        r'DBQ=' + database_path + ';'
    )

    # Establish a connection to the Access database
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Get a list of all tables in the database
    tables = [row.table_name for row in cursor.tables(tableType='TABLE')]

    # Loop through the tables and load each into a DataFrame
    for table in tables:
        query = f'SELECT * FROM [{table}]'
        df = pd.read_sql(query, conn)
        globals()[f'tbl_{table}'] = df  # Create a global variable with the table name

    # Close the connection
    conn.close()

def update_access_table(database_path, table_name, df):
    # Connection string for Access database
    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        r'DBQ=' + database_path + ';'
    )

    # Establish a connection to the Access database
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Clear the existing table
    cursor.execute(f'DELETE FROM [{table_name}]')
    conn.commit()

    # Insert the updated data
    for index, row in df.iterrows():
        columns = ', '.join(row.index)
        placeholders = ', '.join(['?' for _ in row])
        values = tuple(row)
        sql = f'INSERT INTO [{table_name}] ({columns}) VALUES ({placeholders})'
        cursor.execute(sql, values)
    conn.commit()

    # Close the connection
    conn.close()

# Paths
original_database_path = 'E:\\digidure\\CLERUS_v1_DRC_07082024.accdb'
temp_database_path = 'E:\\digidure\\CLERUS_v2_DRC_DM_'+formatted_date+'.accdb'

# Copy the database
copy_database(original_database_path, temp_database_path)

# Export data from the copied database
export_access_to_dataframes(temp_database_path)

In [ ]:
# Function to append the data to access database
def append_to_access_table(df, database_path, table_name):
    """
    Appends a DataFrame to a table in an Access database.

    Parameters:
    df (pandas.DataFrame): The DataFrame to append.
    database_path (str): The path to the Access database file.
    table_name (str): The name of the table to append the data to.
    """
    # Connection string for Access database
    connection_string = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        r'DBQ=' + database_path + ';'
    )

    # Connect to the Access database
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()

    # Append DataFrame to the Access table
    for index, row in df.iterrows():
        columns = ', '.join(row.index)
        values = ', '.join(['?' for _ in row])
        sql = f'INSERT INTO {table_name} ({columns}) VALUES ({values})'
        cursor.execute(sql, tuple(row))

    conn.commit()
    cursor.close()
    conn.close()

In [ ]:
# Panda settings for showing data (this is foremost done to more easily explore the data while processing it)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
# All the individuals that could be matched with DRC are already curated and processed. The next step is to extract individuals from DM that are not in DRC.
# When a link between DM and DRC was found, a value for clerus_id or new_clerus_id was produced. Thus to isolate those that have not been matched we create
# a subselection existing of rows where these fields are empty.
filtered_only_DM = tbl_999_Dm_all_drc_match[tbl_999_Dm_all_drc_match['clerus_id'].isna() & tbl_999_Dm_all_drc_match['new_clerus_id'].isna()]


In [ ]:
# Since we want to link the individuals to clerus we want to use the individual_id lateron as a new clerus_id. To avoid overlap in numbers we added 9000000 to every id. The individual ids have been created through 2_1_check_links_DRC_DM.ipynb
filtered_only_DM["individual_id"] = filtered_only_DM["individual_id"] + 9000000

In [ ]:
# From this dataset we will be extracting all kind of information to align with the Clerus database structure.
# First we extract information for every individual which we use to fill in the table 01_clerus_bio. These are the names of the individual, the year of death and remarks (Bijzonderheden).
# since we are not interested in all the other fields we make a subselection

DM_tbl_bio_1 = filtered_only_DM[['pid', 'predikant', "vertrek naar of vanwege", "jaar vertrek", "Bijzonderheden", "individual_id"]]

In [ ]:
# Extract Year of Death
# After analysing the data in DM it appears that information about someone's year of death can be found in the field "Bijzonderheden", but also in the field "vertrek naar of vanwege".
# To extract this information and to save it in a new field is done in two steps.

# first we extract year from Bijzonderheden where it contains the string "overl." or "overleden"
def extract_year(bijzonderheden):
    if isinstance(bijzonderheden, str):
        match = re.search(r'(overleden|overl\.)\s*(\d{4})', bijzonderheden, re.IGNORECASE)
        if match:
            return int(match.group(2))
    return None

DM_tbl_bio_1['death_year'] = DM_tbl_bio_1['Bijzonderheden'].apply(extract_year)

# Second we update 'death_year' where it is None, using 'jaar_vertrek' if 'vertrek_naar_of_vanwege' contains 'overleden' or 'overl.'
DM_tbl_bio_1['death_year'] = DM_tbl_bio_1.apply(
    lambda row: row['jaar vertrek']
    if pd.isnull(row['death_year']) and
       isinstance(row['vertrek naar of vanwege'], str) and
       ('overleden' in row['vertrek naar of vanwege'].lower() or 'overl.' in row['vertrek naar of vanwege'].lower())
    else row['death_year'],
    axis=1
)

In [ ]:
# Next we need to make another subselection of the data to get only the name, clerus_id, year of death and remarks.
DM_tbl_bio_2 = DM_tbl_bio_1[['predikant', "Bijzonderheden", "individual_id", "death_year"]]

In [ ]:
DM_tbl_bio_2['death_year'] = DM_tbl_bio_2['death_year'].fillna(0).astype(int)

In [ ]:
# Now we remove all duplicates that remain based on the inidivual_id and prioritize on whether "year_death" has a value.
DM_tbl_bio_2 = DM_tbl_bio_2.sort_values(by='death_year', ascending=False, na_position='last')

In [ ]:
# Drop duplicates based on 'individual_id', keeping the first occurrence
DM_tbl_bio_unique = DM_tbl_bio_2.drop_duplicates(subset='individual_id', keep='first')

In [ ]:
DM_tbl_bio_unique[['surname', 'first_name']] = DM_tbl_bio_unique['predikant'].str.split(';', expand=True)
DM_tbl_bio_unique.drop(columns=['predikant'], inplace=True)

In [ ]:
DM_tbl_bio_unique.rename(columns={'Bijzonderheden': 'remarks', 'individual_id': 'clerus_id'}, inplace=True)

In [ ]:
DM_tbl_bio_unique['remarks_source'] = "DM"

In [ ]:
#Add information about the sex to the individual in DM. All are male except when the name contains "mevr" or "mevrouw"

def determine_sex(first_name):
    if first_name and 'mevr' in first_name.lower():
        return 'female'
    else:
        return 'male'

In [ ]:
DM_tbl_bio_unique['sex'] = DM_tbl_bio_unique['first_name'].apply(determine_sex)

In [ ]:
infixes = [ " ter"," ten", " van der", " van den", " van", " den", " der", " de",
            ".ter",".ten", ".van der", ".van den", ".van", ".den", ".der", ".de"]

In [ ]:
def extract_infix(name, infixes):
    if name is None:
        return None
    for infix in infixes:
        if infix in name:
            return infix
    return None

In [ ]:
DM_tbl_bio_unique['infix'] = DM_tbl_bio_unique['first_name'].apply(lambda x: extract_infix(x, infixes))
DM_tbl_bio_unique['infix'] = DM_tbl_bio_unique['infix'].str.replace('.', '', regex=False)

In [ ]:
def remove_infix(name, infix):
    if name and infix:
        return name.replace(infix, '').strip()
    return name

DM_tbl_bio_unique['first_name'] = DM_tbl_bio_unique.apply(lambda row: remove_infix(row['first_name'], row['infix']), axis=1)

In [ ]:
# Append data to database
table_name = '01_clerus_bio'
append_to_access_table(DM_tbl_bio_unique, temp_database_path, table_name)

In [ ]:
bio_death = DM_tbl_bio_unique[['clerus_id','death_year']]
bio_death.rename(columns={'death_year': 'temp_death_year_DM'}, inplace=True)

In [ ]:
# Get roles seperate out of the dataframe where roles from individuals that are only present in DM are used. (i.e. filtered_only_DM)
# Provincie [role_province], Classis [role_classis], Gemeente [role_place], clerus_id, pid(as backup), jaar intrede [year_start], jaar vertrek [year_end], dag intrede - maand intrede - jaar intrede [role_start_accuracy], dag vertrek - maand vertrek - jaar vertrek [role_end_accuracy]

filtered_only_DM.head()

In [ ]:

role_dm = filtered_only_DM

In [ ]:
# from the day, month and year create a date field which we will fill in to start_date_accuracy and end_date_accuracy
role_dm['role_start_date_exact'] = role_dm['dag intrede'].astype(str) + " " + role_dm['maand intrede'] + " " + role_dm['jaar intrede'].astype(str)
role_dm['role_end_date_exact'] = role_dm['dag vertrek'].astype(str) + " " + role_dm['maand vertrek'] + " " + role_dm['jaar vertrek'].astype(str)

In [ ]:
role_dm.drop(columns=['ind_id','clerus_id', 'new_clerus_id', 'dag vertrek', 'maand vertrek', 'dag intrede', 'maand intrede', 'vertrek naar of vanwege', 'Herkomst'], inplace=True)

In [ ]:
role_dm.rename(columns={'Bijzonderheden': 'role_remarks',
                        'individual_id': 'clerus_id',
                        'pid':'role_source_id',
                        'provincie': 'role_province',
                        'classis': 'role_classis',
                        'gemeente': 'role_place',
                        'wijk': 'role_parish',
                        'jaar intrede': 'role_start_year',
                        'jaar vertrek': 'role_end_year',
                        'predikant':'temp_name_surname'}, inplace=True)

In [ ]:
role_dm['role_type'] = "predikant"
role_dm['role_remarks_source'] = "DM"

In [ ]:
role_dm['role_end_year'] = role_dm['role_end_year'].fillna(0).astype(int)

In [ ]:
role_dm = role_dm.replace({np.nan: ''})

In [ ]:
role_dm_death = pd.merge(role_dm, bio_death, on='clerus_id', how='left')

In [ ]:
table_name = '12_clerus_role'
append_to_access_table(role_dm_death, temp_database_path, table_name)